<a href="https://colab.research.google.com/github/Devauntae/AI-ML-projects/blob/tradingBOT/TradingBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install lumibot==2.9.13 --force-reinstall
!pip install pandas==2.2.2 scipy>=1.11.2 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 55.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 137.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.5/825.5 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 134.0 MB/s eta 0:00:00
   ━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lumibot 2.9.13 requires pandas<=2.0.3,>=2.0.0, but you have pandas 2.2.2 which is incompatible.
lumibot 2.9.13 requires scipy==1.10.1, but you have scipy 1.15.2 which is incompatible.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.32.0 which is incompatible.
ibis-framework 9.2.0 requires toolz<1,>=0.11, but you have toolz 1.0.0 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.3 which is incompatible.
moviepy 1.0.3 requires decorator<5.0,>=4.0.2, but you have decorator 5.2.1 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.3 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.
pytensor 2.27.1 requires numpy<2,>=1.17.0, but you have numpy 2.2.3 which

In [3]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import trader
from datetime import datetime
from alpaca_trade_api.rest import REST
from timedelta import timedelta
from finbert_utils import estimate_sentiment

ValueError: Invalid frequency: D, failed to parse with error message: TypeError("Invalid type for timedelta scalar: <class 'numpy.timedelta64'>")

In [ ]:
API_KEY = "PKB9JQZLXJB7D3N4I2D9"
API_SECRET = "kbN1CQwZssOQOnkEq55UshrHvgDzMxKff9ZvQCwI"
BASE_URL = "https://paper-api.alpaca.markets/v2"

ALPACA_CREDS = {
    "API_KEY":API_KEY,
    "API_SECRET":API_SECRET,
    "PAPER": True
}

In [ ]:
class MLTrader(Strategy):
  def initialize(self, symbol:str="SPY", cash_at_risk:float=.5):
    self.symbol = symbol
    self.sleeptime = "24H"
    self.last_trade = None
    self.cash_at_risk = cash_at_risk
    self.api = REST(key_id=API_KEY, secret_key=API_SECRET, base_url=BASE_URL)

  def position_sizing(self):
      cash = self.get_cash()
      last_price = self.get_last_price(self.symbol)
      quantity = round(cash * self.cash_at_risk / last_price,0)
      return cash, last_price, quantity

  def get_dates(self):
    today = self.get_datetime()
    three_days_prior = today - timedelta(days=3)
    return today.strfttime('%Y-%m-%d'), three_days_prior.strfttime('%Y-%m-%d')

  def get_sentiment(self):
    today, three_days_prior = self.get_dates()
    news = self.api.get_news(symbool=self.symbol,
                             start=three_days_prior,
                             end=today)
    news = [ev.__dict__["_raw"]["headline"] for ev in news]
    probability, sentiment = estimate_setiment(news)
    return probability, sentiment


  def on_trading_iteration(self):
    cash, last_price, quantity = self.position_sizing()
    probability, sentiment = self.get_sentiment()

    if cash > last_price:
      if sentiment == "positive" and probability > .999:
        if self.last_order == "sell":
          self.sell_all()
        order = self.create_order(
            self.symbol,
            quantity,
            "buy",
            type = "bracket",
            take_profit_price = last_price*1.20,
            stop_loss_price = last_price*.95
        )
        self.submit_order(order)
        self.last_trade = "buy"

      elif sentiment == "negative" and probability > .999:
        if self.last_order == "buy":
          self.sell_all()
        order = self.create_order(
            self.symbol,
            quantity,
            "sell",
            type = "bracket",
            take_profit_price = last_price*.9,
            stop_loss_price = last_price*1.05
        )
        self.submit_order(order)
        self.last_trade = "sell"


start_date = datetime(2025,2,1)
end_date = datetime(2025,3,1)

broker = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name ='mlstrat', broker=broker, parameters={"symbol":"SPY","cash_at_risk":.5})
strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol":"SPY","cash_at_risk":.5}
)
